**Fluorescence Microscopy Image Processor Notebook**
----------------------------------------------------

This Jupyter Notebook provides a set of operations to process fluorescence microscopy images using a false coloring scheme. The processes include reading in nuclear and cytoplasmic channels from an HDF5 file, applying false HE coloring to them, and saving the results as TIFF images. The approach uses the method described in the paper: [Serafin et al. 2020](https://doi.org/10.1371/journal.pone.0233198).

**Dependencies**:

- Custom `falsecolor` module for false coloring operations.
- Image reading and writing libraries such as `h5py` and `tifffile`.
- Standard Python libraries for data processing and manipulation.
- ImageJ or Fiji software.

Author: takakiom  
Date: 09/19/2023  
License: MIT License (refer to the LICENSE file in the root directory for more details)


**Setting Parameters**:
Before performing image operations, we set paths, parameters for coloring, and other essential settings.


In [ ]:
import falsecolor.coloring as fc
from falsecolor.process import ViewImage
import numpy as np
import os
import h5py as h5
import glob
import time
import datetime
import tifffile
from multiprocessing import Pool

**Image Processing**:
This section involves reading in the nuclear and cytoplasmic channels, applying false coloring, and saving the processed images.

You can create a HDF5 file easily using ImageJ/FIJI software with BigStitcher plugin.


In [ ]:
###
# Set params
###

# Path for the target hdf file
hdf_path = '/enter/your/hdf5//dataset.h5'
base_path = os.path.dirname(hdf_path)

# Set param for coloring
# Modify these params to meet your applications
param_nuc = 200
param_cyto = 1600


# Base path  for output data
date = datetime.date.today()
date= date.strftime("%m%d")
base_outpath = os.path.join(base_path, f'output_{date}')
os.makedirs(base_outpath, exist_ok=True)


In [ ]:
with h5.File(hdf_path,'r') as f:
    
    #read nuclear channel into memory
    nuclei_HE = f['t00000/s00/0/cells'][:]
    
    #read cytoplasmic channel into memory
    cyto_HE = f['t00000/s01/0/cells'][:]
f.close()

print(nuclei_HE.shape, cyto_HE.shape)

#grab color settings for virtual H&E
HE_settings = fc.getColorSettings(key = 'HE')

#Color settings for each image are in RGB order
print(HE_settings)

# Get the directory name containing the file
dir_name = os.path.dirname(hdf_path)
# Get the last part of the path (the folder name)
folder_name = os.path.split(dir_name)[-1]

# full tif saving

# Create the directory if it doesn't exist
single_outpath = os.path.join(base_outpath, 'sigle_slices' )
os.makedirs(single_outpath, exist_ok=True)

stack = None

if nuclei_HE.shape != cyto_HE.shape :
    raise ValueError("Different shape")
slices = nuclei_HE.shape[0]

for i in range(0,slices,200):
    #default normalization levels for non-flatfielded data are set as follows
    TestHE = fc.rapidFalseColor(nuclei_HE[i], cyto_HE[i], HE_settings['nuclei'], HE_settings['cyto'],
                                    nuc_normfactor = param_nuc, cyto_normfactor = param_cyto)
    if stack is None:
        stack = np.array([TestHE])
    else:
        stack = np.append(stack, [TestHE], axis=0)
    
    # For each slice
    title_single = f'{folder_name}_slice#{i}.tif'
    # Save the figure as a high-resolution TIFF image
    out_path_single = os.path.join(single_outpath, title_single )
    # Assuming img is your image data in a numpy array
    tifffile.imwrite(out_path_single, TestHE)

# Save the figure as a TIFF stack
out_path_stack = os.path.join(base_outpath, "FluroHE_stack.tif" )
# Assuming img is your image data in a numpy array
tifffile.imwrite(out_path_stack, stack)

(Optional)**Multiprocessing for Efficient Image Processing**:
To speed up the image processing operations, we utilize multiple CPU cores to process slices of the microscopy data concurrently.


(Optional)**Multiprocessing**

In [ ]:
%%time

processes = 8

def process_slice(i):
    if np.all(nuclei_HE[i] == 0) and np.all(cyto_HE[i] == 0):
        return None
    else:
        TestHE = fc.rapidFalseColor(nuclei_HE[i], cyto_HE[i], HE_settings['nuclei'], HE_settings['cyto'],
                                    nuc_normfactor = param_nuc, cyto_normfactor = param_cyto)
        return TestHE

def save_slice(data, i):
    if data is None:
        return
    title_single = f'{folder_name}_slice#{i}.tif'
    out_path_single = os.path.join(single_outpath, title_single )
    tifffile.imwrite(out_path_single, data)

with h5.File(hdf_path,'r') as f:
    nuclei_HE = f['t00000/s00/0/cells'][:]
    cyto_HE = f['t00000/s01/0/cells'][:]

print(nuclei_HE.shape, cyto_HE.shape)

HE_settings = fc.getColorSettings(key = 'HE')
print(HE_settings)

dir_name = os.path.dirname(hdf_path)
folder_name = os.path.split(dir_name)[-1]

single_outpath = os.path.join(base_outpath, 'sigle_slices' )
os.makedirs(single_outpath, exist_ok=True)

if nuclei_HE.shape != cyto_HE.shape :
    raise ValueError("Different shape")

slices = nuclei_HE.shape[0]
slice_indices = list(range(50, 650))

with Pool(processes=processes) as pool:
    results = pool.map(process_slice, slice_indices)

results = [r for r in results if r is not None]
stack = np.stack(results, axis=0)

out_path_stack = os.path.join(base_outpath, "FluroHE_stack.tif" )
tifffile.imwrite(out_path_stack, stack)

with Pool(processes=processes) as pool:
    pool.starmap(save_slice, zip(results, slice_indices))


**Final Remarks**:
- Always ensure you have the necessary permissions to access and process the data.
- Please refer to the aforementioned paper for details on the false coloring method used in this notebook.
